# Fixing EchoTimes

In this section we verify that the subjects with the `ECHO_TIME` errors have missing fieldmaps. Because these subjects were copied from PNC we will verify in that section that there are no fieldmaps available, only `phasediff`

In [1]:
import flywheel
import pandas as pd
from fw_heudiconv.cli.export import get_nested

client = flywheel.Client()

/home/ttapera/miniconda3/envs/flywheel/lib/python3.7/site-packages/fw_heudiconv/backend_funcs/query.py:4: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  from nibabel.nicom.dicomwrappers import wrapper_from_data


200910-19:25:19,88 nipype.utils INFO:
	 Running nipype version 1.4.0 (latest: 1.5.0)


INFO: Running nipype version 1.4.0 (latest: 1.5.0)


In [2]:
df = pd.read_csv('/storage/ttapera/RBC/PennLINC/Validation/PNC/validation.csv')
echotimes_rows = df.loc[df['code'] == 15,]

In [3]:
echotimes_rows.subject

317      sub-255089519
452     sub-3260960488
502     sub-3520758666
971     sub-2441231201
1013    sub-3103368139
2890    sub-1163553824
3149    sub-1060668554
3822     sub-398131152
4944    sub-1512248949
6056    sub-1682850665
6119    sub-1867827374
6520    sub-1095340536
6741     sub-134279804
7147    sub-2216595430
7398    sub-3925705705
7420    sub-4066124203
7554     sub-458296344
Name: subject, dtype: object

We're going to confirm that these cases are all problematic because they have no fieldmaps to begin with in the PNC project:

In [4]:
# get the subject ids <-> bblids

rbc_subjects = list(echotimes_rows.subject)
df2 = pd.read_csv('/storage/ttapera/RBC/data/bblids_3.csv', names=['bblid', 'hash', 'fullid', 'process'])

pnc_subjects = df2[df2['fullid'].isin(rbc_subjects)]
pnc_subjects = [str(i) for i in list(pnc_subjects.bblid)]

In [5]:
# do some flywheel stuff

pnc_proj = client.projects.find_first('label=PNC_CS_810336')
subjects = pnc_proj.subjects()
subjects2 = [sub for sub in subjects if sub.label in pnc_subjects]

In [6]:
subjects_dict = dict(zip(pnc_subjects, rbc_subjects))

In [7]:
# print the series description of dicoms available in each subject's acquisitions

for sub in subjects2:
    print(subjects_dict[sub.label])
    print("Available DICOMS:") 
    sessions = sub.sessions()
    for s in sessions:
        acquisitions = s.acquisitions()
        
        series = []
        for acq in acquisitions:
            acq = client.get(acq.id)

            files = [f for f in acq.files if 'dicom' in f.name]
            series.extend([get_nested(f, 'info', 'SeriesDescription') for f in files])
        print(series)
        print()

sub-458296344
Available DICOMS:
['DTI_2x32_36', 'DTI_2x32_35', 'bbl1_frac2back1_231', 'bbl1_idemo2_210', 'epi_singlerep', 'MoCoSeries', 'ep2d_se_pcasl_PHC_1200ms', 'MPRAGE_TI1100_ipat2', 'bbl1_restbold1_124', 'B0map_3.0mm', 'localizer']

sub-3925705705
Available DICOMS:
['DTI_2x32_36', 'DTI_2x32_35', 'bbl1_idemo2_210', 'bbl1_frac2back1_231', 'epi_singlerep', 'MoCoSeries', 'ep2d_se_pcasl_PHC_1200ms', 'MPRAGE_TI1100_ipat2', 'bbl1_restbold1_124', 'B0map_3.0mm', 'localizer']

sub-2441231201
Available DICOMS:
['DTI_2x32_35', 'localizer', 'bbl1_frac2back1_231', 'bbl1_idemo2_210', 'epi_singlerep', 'MoCoSeries', 'ep2d_se_pcasl_PHC_1200ms', 'MPRAGE_TI1100_ipat2', 'bbl1_restbold1_124', 'B0map_3.0mm', 'DTI_2x32_36', 'localizer']

sub-3103368139
Available DICOMS:
['DTI_2x32_36', 'DTI_2x32_35', 'bbl1_frac2back1_231', 'bbl1_idemo2_210', 'epi_singlerep', 'MoCoSeries', 'ep2d_se_pcasl_PHC_1200ms', 'MPRAGE_TI1100_ipat2', 'bbl1_restbold1_124', 'B0map_3.0mm', 'localizer']

sub-2216595430
Available DICOMS:

Indeed, all 17 of these participants only have `B0map_3.0mm`, which is the phasediff file we find, but not magnitude files. Hence, this issue can't be directly "fixed", but we must note these subjects.

It's worth noting that these are the same subjects with the `MISSING MAGNITUDE` error as well:

In [9]:
missing_mag = df.loc[df['code'] == 92,]
missing_mag.subject

314      sub-255089519
449     sub-3260960488
499     sub-3520758666
968     sub-2441231201
1010    sub-3103368139
2887    sub-1163553824
3146    sub-1060668554
3819     sub-398131152
4941    sub-1512248949
6053    sub-1682850665
6116    sub-1867827374
6517    sub-1095340536
6738     sub-134279804
7144    sub-2216595430
7395    sub-3925705705
7417    sub-4066124203
7551     sub-458296344
Name: subject, dtype: object

In [10]:
echotimes_rows.subject

317      sub-255089519
452     sub-3260960488
502     sub-3520758666
971     sub-2441231201
1013    sub-3103368139
2890    sub-1163553824
3149    sub-1060668554
3822     sub-398131152
4944    sub-1512248949
6056    sub-1682850665
6119    sub-1867827374
6520    sub-1095340536
6741     sub-134279804
7147    sub-2216595430
7398    sub-3925705705
7420    sub-4066124203
7554     sub-458296344
Name: subject, dtype: object

In [15]:
set(missing_mag.subject) - set(echotimes_rows.subject)

set()

In [16]:
set(echotimes_rows.subject) - set(missing_mag.subject)

set()